# **Extracting Information from Legal Documents Using RAG**

## **Objective**

The main objective of this assignment is to process and analyse a collection text files containing legal agreements (e.g., NDAs) to prepare them for implementing a **Retrieval-Augmented Generation (RAG)** system. This involves:

* Understand the Cleaned Data : Gain a comprehensive understanding of the structure, content, and context of the cleaned dataset.
* Perform Exploratory Analysis : Conduct bivariate and multivariate analyses to uncover relationships and trends within the cleaned data.
* Create Visualisations : Develop meaningful visualisations to support the analysis and make findings interpretable.
* Derive Insights and Conclusions : Extract valuable insights from the cleaned data and provide clear, actionable conclusions.
* Document the Process : Provide a detailed description of the data, its attributes, and the steps taken during the analysis for reproducibility and clarity.

The ultimate goal is to transform the raw text data into a clean, structured, and analysable format that can be effectively used to build and train a RAG system for tasks like information retrieval, question-answering, and knowledge extraction related to legal agreements.

### **Business Value**  


The project aims to leverage RAG to enhance legal document processing for businesses, law firms, and regulatory bodies. The key business objectives include:

* Faster Legal Research: <br> Reduce the time lawyers and compliance officers spend searching for relevant case laws, precedents, statutes, or contract clauses.
* Improved Contract Analysis: <br> Automatically extract key terms, obligations, and risks from lengthy contracts.
* Regulatory Compliance Monitoring: <br> Help businesses stay updated with legal and regulatory changes by retrieving relevant legal updates.
* Enhanced Decision-Making: <br> Provide accurate and context-aware legal insights to assist in risk assessment and legal strategy.


**Use Cases**
* Legal Chatbots
* Contract Review Automation
* Tracking Regulatory Changes and Compliance Monitoring
* Case Law Analysis of past judgments
* Due Diligence & Risk Assessment

## **1. Data Loading, Preparation and Analysis** <font color=red> [20 marks] </font><br>

### **1.1 Data Understanding**

The dataset contains legal documents and contracts collected from various sources. The documents are present as text files (`.txt`) in the *corpus* folder.

There are four types of documents in the *courpus* folder, divided into four subfolders.
- `contractnli`: contains various non-disclosure and confidentiality agreements
- `cuad`: contains contracts with annotated legal clauses
- `maud`: contains various merger/acquisition contracts and agreements
- `privacy_qa`: a question-answering dataset containing privacy policies

The dataset also contains evaluation files in JSON format in the *benchmark* folder. The files contain the questions and their answers, along with sources. For each of the above four folders, there is a `json` file: `contractnli.json`, `cuad.json`, `maud.json` `privacy_qa.json`. The file structure is as follows:

```
{
    "tests": [
        {
            "query": <question1>,
            "snippets": [{
                    "file_path": <source_file1>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 1>
                },
                {
                    "file_path": <source_file2>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 2>
                }, ....
            ]
        },
        {
            "query": <question2>,
            "snippets": [{<answer context for que 2>}]
        },
        ... <more queries>
    ]
}
```

### **1.2 Load and Preprocess the data** <font color=red> [5 marks] </font><br>

#### Loading libraries

In [11]:
## The following libraries might be useful
!pip install -q langchain-openai
!pip install -U -q langchain-community
!pip install -U -q langchain-chroma
!pip install -U -q datasets
!pip install -U -q ragas
!pip install -U -q rouge_score

In [12]:
# Import essential libraries

import numpy as np
import os
import json
import random
import re
import glob
import logging
from collections import Counter

# Data Handling & Analysis
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# NLP Specific Libraries
import nltk
from nltk.corpus import stopwords
from transformers import AutoTokenizer

# Database/Vector Store Specific Imports
import chromadb
from chromadb.config import Settings # Specific ChromaDB settings


In [13]:
# LangChain Specific Imports
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain.chains import RetrievalQA

import warnings
warnings.filterwarnings('ignore')

#### **1.2.1** <font color=red> [3 marks] </font>
Load all `.txt` files from the folders.

You can utilise document loaders from the options provided by the LangChain community.

Optionally, you can also read the files manually, while ensuring proper handling of encoding issues (e.g., utf-8, latin1). In such case, also store the file content along with metadata (e.g., file name, directory path) for traceability.

In [16]:
# Load the files as documents

from pathlib import Path

def safe_text_loader(file_path):
    for enc in ["utf-8", "cp1252", "latin-1"]:
        try:
            return TextLoader(file_path, encoding=enc).load()
        except UnicodeDecodeError:
            continue
    return []

corpus_root = Path(r"C:\Users\Saravana\OneDrive\Desktop\RAG assessment\rag_legal\corpus")
subfolders = ["contractnli", "cuad", "maud", "privacy_qa"]

documents_by_category = {
    folder: sum([safe_text_loader(str(f)) for f in (corpus_root / folder).rglob("*.txt")], [])
    for folder in subfolders
}

for folder, docs in documents_by_category.items():
    print(f"Loaded {len(docs)} documents from '{folder}'")

all_documents = sum(documents_by_category.values(), [])
print(f"Total documents loaded: {len(all_documents)}")

Loaded 95 documents from 'contractnli'
Loaded 462 documents from 'cuad'
Loaded 134 documents from 'maud'
Loaded 7 documents from 'privacy_qa'
Total documents loaded: 698


#### **1.2.2** <font color=red> [2 marks] </font>
Preprocess the text data to remove noise and prepare it for analysis.

Remove special characters, extra whitespace, and irrelevant content such as email and telephone contact info.
Normalise text (e.g., convert to lowercase, remove stop words).
Handle missing or corrupted data by logging errors and skipping problematic files.

In [19]:
# Clean and preprocess the data

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

logging.basicConfig(filename='preprocessing_errors.log', level=logging.ERROR)

def clean_text(text):
    try:
        # Remove emails and phone numbers
        text = re.sub(r'\S+@\S+|\b\d{10,}\b', '', text)
        # Remove special characters and digits
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        # Lowercase and remove extra whitespace
        text = re.sub(r'\s+', ' ', text.lower()).strip()
        # Remove stopwords
        return ' '.join(word for word in text.split() if word not in stop_words)
    except Exception as e:
        logging.error(f"Error cleaning text: {e}")
        return ""

# Preprocess all loaded documents
cleaned_documents = []
for doc in all_documents:
    try:
        cleaned = clean_text(doc.page_content)
        if cleaned:
            cleaned_documents.append(cleaned)
    except Exception as e:
        logging.error(f"Error processing document: {e}")

print(f"Cleaned and kept {len(cleaned_documents)} of {len(all_documents)} documents.")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Saravana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Cleaned and kept 698 of 698 documents.


### **1.3 Exploratory Data Analysis** <font color=red> [10 marks] </font><br>

#### **1.3.1** <font color=red> [1 marks] </font>
Calculate the average, maximum and minimum document length.

In [22]:
# Calculate the average, maximum and minimum document length.

word_counts = [len(doc.split()) for doc in cleaned_documents]
print(f"Average Length: {sum(word_counts)/len(word_counts):.2f}")
print(f"Maximum Length: {max(word_counts)}")
print(f"Minimum Length: {min(word_counts)}")

Average Length: 8621.62
Maximum Length: 82546
Minimum Length: 132


#### **1.3.2** <font color=red> [4 marks] </font>
Analyse the frequency of occurrence of words and find the most and least occurring words.

Find the 20 most common and least common words in the text. Ignore stop words such as articles and prepositions.

In [25]:
# Find frequency of occurence of words

all_words = ' '.join(cleaned_documents).split()
word_freq = Counter(all_words)
# Top 20 most common words
most_common = word_freq.most_common(20)
print("Most Common Words:")
for word, freq in most_common:
    print(f"{word}: {freq}")

# Top 20 least common words
least_common = word_freq.most_common()[:-21:-1]
print("\nLeast Common Words:")
for word, freq in least_common:
    print(f"{word}: {freq}")

Most Common Words:
company: 148175
shall: 107998
agreement: 104577
section: 75368
parent: 58010
party: 49664
date: 39300
time: 35256
material: 34211
merger: 33848
subsidiaries: 33318
b: 31719
applicable: 31370
including: 29399
respect: 28848
may: 28114
stock: 26654
information: 25687
parties: 24614
business: 23531

Least Common Words:
newer: 1
peoplefuncom: 1
nonmarketing: 1
httpwwwyouronlinechoiceseu: 1
httpwwwaboutadsinfochoices: 1
checkins: 1
httpsvunglecomprivacypolicy: 1
vungle: 1
httpswwwtapresearchcomuserprivacy: 1
tapresearch: 1
httpsdevtapjoycomfaqtapjoyprivacypolicy: 1
tapjoy: 1
httpswwwstartappcompolicyprivacypolicy: 1
startappcom: 1
httpaboutsoomlaenduserprivacypolicy: 1
soomla: 1
httpswwwsmaatocomprivacy: 1
smaato: 1
httpspinsightmediacomprivacy: 1
pinsight: 1


#### **1.3.3** <font color=red> [4 marks] </font>
Analyse the similarity of different documents to each other based on TF-IDF vectors.

Transform some documents to TF-IDF vectors and calculate their similarity matrix using a suitable distance function. If contracts contain duplicate or highly similar clauses, similarity calculation can help detect them.

Identify for the first 10 documents and then for 10 random documents. What do you observe?

In [28]:
# Transform the page contents of documents
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(cleaned_documents)
# Compute similarity scores
similarity_matrix = cosine_similarity(tfidf_matrix)

In [29]:
# create a list of 10 random integers
random_indices = random.sample(range(len(cleaned_documents)), 10)
print("Random Indices:", random_indices)

Random Indices: [287, 207, 239, 505, 22, 634, 161, 637, 236, 365]


In [30]:
# Compute similarity scores for 10 random documents
similarity_random_10 = cosine_similarity(tfidf_matrix[random_indices])
print(similarity_random_10)


[[1.         0.06856943 0.04731035 0.12014691 0.03712537 0.10639599
  0.07169182 0.13424613 0.06103542 0.02407345]
 [0.06856943 1.         0.04652375 0.12859128 0.03976421 0.07583958
  0.05055198 0.09677515 0.03735938 0.02103475]
 [0.04731035 0.04652375 1.         0.08057317 0.02031859 0.06709876
  0.03901286 0.08532064 0.02589319 0.01267901]
 [0.12014691 0.12859128 0.08057317 1.         0.09888746 0.13567078
  0.09458191 0.16286126 0.07887172 0.03370378]
 [0.03712537 0.03976421 0.02031859 0.09888746 1.         0.07577898
  0.03861083 0.10630318 0.02721656 0.01025782]
 [0.10639599 0.07583958 0.06709876 0.13567078 0.07577898 1.
  0.19043271 0.74414589 0.09408205 0.02407516]
 [0.07169182 0.05055198 0.03901286 0.09458191 0.03861083 0.19043271
  1.         0.24619616 0.06089699 0.01539593]
 [0.13424613 0.09677515 0.08532064 0.16286126 0.10630318 0.74414589
  0.24619616 1.         0.13243589 0.0308685 ]
 [0.06103542 0.03735938 0.02589319 0.07887172 0.02721656 0.09408205
  0.06089699 0.13243

### **1.4 Document Creation and Chunking** <font color=red> [5 marks] </font><br>

#### **1.4.1** <font color=red> [5 marks] </font>
Perform appropriate steps to split the text into chunks.

In [33]:
# Process files and generate chunks
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1. Wrap cleaned strings into Document objects
docs = [Document(page_content=doc) for doc in cleaned_documents]

# 2. Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)

# 3. Split the documents into chunks
chunks = text_splitter.split_documents(docs)

# 4. Display the number of chunks created
print(f"Number of chunks created: {len(chunks)}")


Number of chunks created: 198215


## **2. Vector Database and RAG Chain Creation** <font color=red> [15 marks] </font><br>

### **2.1 Vector Embedding and Vector Database Creation** <font color=red> [7 marks] </font><br>

#### **2.1.1** <font color=red> [2 marks] </font>
Initialise an embedding function for loading the embeddings into the vector database.

Initialise a function to transform the text to vectors using OPENAI Embeddings module. You can also use this function to transform during vector DB creation itself.

In [38]:
# Fetch your OPENAI API Key as an environment variable
import os

# Set your GROQ API key here
os.environ["GROQ_API_KEY"] = "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"

In [94]:
# Initialise an embedding function
import torch

embedding_function = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
)

print("Embedding model device:", "cuda" if torch.cuda.is_available() else "cpu")

Embedding model device: cuda


#### **2.1.2** <font color=red> [5 marks] </font>
Load the embeddings to a vector database.

Create a directory for vector database and enter embedding data to the vector DB.

In [42]:
# Install faiss-cpu
!pip install faiss-cpu

In [43]:
# Add Chunks to vector DB
vector_db = FAISS.from_documents(all_documents, embedding_function)

# save the vector database locally
vector_db.save_local("vector_db")

In [44]:
retriever = vector_db.as_retriever(search_kwargs={"k": 1})

### **2.2 Create RAG Chain** <font color=red> [8 marks] </font><br>

#### **2.2.1** <font color=red> [5 marks] </font>
Create a RAG chain.

In [47]:
# Create a RAG chain
from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="llama3-8b-8192",   # Other Groq-supported models: llama3-70b-8192, mixtral-8x7b-32768
    temperature=0,
    max_tokens=1024,
    max_retries=3,
    timeout=60,
)

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Options: "stuff", "map_reduce", "refine", "map_rerank"
    retriever=retriever,
    return_source_documents=True  # Optional: Show where the answer came from
)

#### **2.2.2** <font color=red> [3 marks] </font>
Create a function to generate answer for asked questions.

Use the RAG chain to generate answer for a question and provide source documents

In [50]:
# Create a function for question answering
def generate_answer(question,qa_chain):
    """
    Generates an answer for the given question using the RAG pipeline (RetrievalQA).
    Args:
        question (str): User's query
    Returns:
        dict: {
            "answer": Generated answer string,
            "source_documents": List of Document objects used for answering
        }
    """
    response = qa_chain(question)
    # Extract answer
    answer = response.get('result', 'No answer generated.')
    # Extract actual Document objects (not just source paths)
    source_documents = response.get('source_documents', [])
    return {"answer": answer,"source_documents": source_documents }


In [51]:
# Example question
# question ="Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?"
question = (
    "Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; "
    "Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?"
)

# Get answer + sources
response = generate_answer(question,qa_chain)

# Extract answer and sources
answer = response["answer"]
sources = response["source_documents"]

# Print the answer
print("Answer:", answer)

# Print full content from source documents (limit to 3 documents, no word limit)
print("\nSource Documents:")
for i, doc in enumerate(sources[:3], 1):
    print(f"\nSource {i} (Metadata: {doc.metadata}):")
    print(doc.page_content[:400].strip() + "...") 

Answer: According to the document, the answer is yes. Clause 5 of the Agreement states: "All materials and Confidential Information shall remain the exclusive property of the Disclosing Party. Except as expressly provided herein or under a separate written agreement between the parties that references this Agreement, either party shall not be obligated to grant, convey or transfer to the other any interest, license or other right, or under its Confidential Information or any patent, copyright, trade secret, trademark or other intellectual property right."

Source Documents:

Source 1 (Metadata: {'source': 'C:\\Users\\Saravana\\OneDrive\\Desktop\\RAG assessment\\rag_legal\\corpus\\contractnli\\TabunKitchenInvestments-NDA.txt'}):
MUTUAL NON-DISCLOSURE & NON-CIRCUMVENTION AGREEMENT
DATE:
THIS NON-DISCLOSURE & NON-CIRUMVENTION AGREEMENT (the “Agreement”) is made by and between
PARTY 1: Tabun Kitchen of 151 Copse Hill, London, SW20 0SU
PARTY 2:
and binding on directors and shareholders unde

## **3. RAG Evaluation** <font color=red> [10 marks] </font><br>

### **3.1 Evaluation and Inference** <font color=red> [10 marks] </font><br>

#### **3.1.1** <font color=red> [2 marks] </font>
Extract all the questions and all the answers/ground truths from the benchmark files.

Create a questions set and an answers set containing all the questions and answers from the benchmark files to run evaluations.

In [56]:
# Create a question set by taking all the questions from the benchmark data
import os
import json
import glob

#Get all JSON files from the flat benchmark folder
directory_path = r"C:\Users\Saravana\OneDrive\Desktop\RAG assessment\rag_legal\benchmarks"
json_files = glob.glob(os.path.join(directory_path, "*.json"))

# Also create a ground truth/answer set
def extract_questions_and_answers(json_files):
    questions = []
    ground_truths = []

    for file in json_files:
        with open(file, "r", encoding="utf-8") as f:
            data = json.load(f)

        for test in data.get("tests", []):
            question = test.get("query")
            answer = test["snippets"][0].get("answer", "") if test.get("snippets") else ""
            questions.append(question)
            ground_truths.append(answer)

    return questions, ground_truths


#Using the function
questions, ground_truths = extract_questions_and_answers(json_files)

# Printing samples
print("Total questions extracted:", len(questions))
print("Sample question:", questions[0])
print("Sample ground truth:", ground_truths[0])


Total questions extracted: 6889
Sample question: Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?
Sample ground truth: Any and all proprietary rights, including but not limited to rights to and in inventions, patent rights, utility models, copyrights, trademarks and trade secrets, in and to any Confidential Information shall be and remain with the Participants respectively, and Mentor shall not have any right, license, title or interest in or to any Confidential Information, except the limited right to review, assess and help develop such Confidential Information in connection with the Copernicus Accelerator 2017.


#### **3.1.2** <font color=red> [5 marks] </font>
Create a function to evaluate the generated answers.

Evaluate the responses on *Rouge*, *Ragas* and *Bleu* scores.

In [59]:
# required libraries
from tqdm import tqdm

# Ragas imports
from ragas.metrics import faithfulness, answer_relevancy
from ragas import evaluate as ragas_evaluate
from datasets import Dataset

# ROUGE and BLEU
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu,SmoothingFunction
from nltk.tokenize import word_tokenize

# LangChain for RAG chain
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq


In [60]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Saravana\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [61]:
# Function to evaluate the RAG pipeline

def evaluate_rag_pipeline(questions, ground_truths, generate_answer, num_questions=2):
    generated_answers = []
    rouge_scores = []
    bleu_scores = []

    smoothie = SmoothingFunction().method4
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

    print(f"Evaluating on first {num_questions} questions...\n")

    for i in tqdm(range(num_questions)):
        question = questions[i]
        ground_truth = ground_truths[i]

        # Step 1: Generate answer from RAG
        response = generate_answer(question)
        predicted = response["answer"]

        generated_answers.append({
            "question": question,
            "ground_truth": ground_truth,
            "answer": predicted,
            "contexts": [doc.page_content for doc in response["source_documents"]]
        })

        # Step 2: ROUGE
        rouge = scorer.score(ground_truth, predicted)
        rouge_scores.append({
            "rouge1": rouge["rouge1"].fmeasure,
            "rougeL": rouge["rougeL"].fmeasure
        })

        # Step 3: BLEU with smoothing
        reference = word_tokenize(ground_truth.lower())
        candidate = word_tokenize(predicted.lower())
        bleu = sentence_bleu([reference], candidate, smoothing_function=smoothie)
        bleu_scores.append(bleu)

    # Prepare dataset for RAGAS
    ragas_data = Dataset.from_list(generated_answers)

    # Only run RAGAS on a small subset (you can expand once verified)
    ragas_result = ragas_evaluate(
        ragas_data.select(range(min(num_questions, 2))),
        metrics=[faithfulness, answer_relevancy],
        llm=llm,
        embeddings=embedding_function
    )

    return {
        "generated": generated_answers,
        "avg_bleu": sum(bleu_scores) / len(bleu_scores),
        "avg_rouge1": sum([r["rouge1"] for r in rouge_scores]) / len(rouge_scores),
        "avg_rougeL": sum([r["rougeL"] for r in rouge_scores]) / len(rouge_scores),
        "ragas": ragas_result.to_pandas()
    }


#### **3.1.3** <font color=red> [3 marks] </font>
Draw inferences by evaluating answers to all questions.

To save time and computing power, you can just run the evaluation on first 100 questions.

In [64]:
# Evaluate the RAG pipeline
# Run evaluation on a small sample (e.g., first 3 questions)
result = evaluate_rag_pipeline(
    questions,
    ground_truths,
    lambda q: generate_answer(q, qa_chain),
    num_questions=100
)

# Print numeric evaluation metrics
print("\n--- Evaluation Metrics ---")
print(f"Average BLEU Score     : {result['avg_bleu']:.4f}")
print(f"Average ROUGE-1 Score  : {result['avg_rouge1']:.4f}")
print(f"Average ROUGE-L Score  : {result['avg_rougeL']:.4f}")

# Print RAGAS scores (only numeric values)
print("\n--- RAGAS Answer Relevancy Scores ---")
print(result["ragas"]["answer_relevancy"].round(4).to_list())

Evaluating on first 100 questions...



100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [33:42<00:00, 20.22s/it]


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]


--- Evaluation Metrics ---
Average BLEU Score     : 0.0754
Average ROUGE-1 Score  : 0.2993
Average ROUGE-L Score  : 0.2174

--- RAGAS Answer Relevancy Scores ---
[0.4849, 0.5586]


## **4. Conclusion** <font color=red> [5 marks] </font><br>

### **4.1 Conclusions and insights** <font color=red> [5 marks] </font><br>

#### **4.1.1** <font color=red> [5 marks] </font>
Conclude with the results here. Include the insights gained about the data, model pipeline, the RAG process and the results obtained.

# 4. Conclusion

## **Data and Preprocessing**
* Legal documents were cleaned and split into smaller parts (chunks) for better processing.
* Each chunk was 300 characters long with 50 characters overlapping to keep context.
* Embeddings were created using `all-MiniLM-L6-v2`, and GPU was used if available.

## **Model Pipeline**
* A Retrieval-Augmented Generation (RAG) setup was built using the `llama3-8b-8192` model through Groq.
* Chroma was used to store and search document chunks.
* The model was set to retrieve only 1 relevant chunk (`k=1`) for answering each question.
* The `"stuff"` method was used to combine context and generate answers.

## **Evaluation Results**
* **BLEU Score**: 0.0754 → The answers had low word match with correct answers.
* **ROUGE-1**: 0.2993 → Some overlap in single words with correct answers.
* **ROUGE-L**: 0.2174 → Less similarity in sentence structure.
* **Answer Relevancy (RAGAS)**: [0.4849, 0.5586] → Answers were somewhat related but not always complete.

## **Insights**
* The scores show that the answers were sometimes useful but not always accurate.
* Using only 1 chunk (`k=1`) may have limited the information the model had to work with.
* The model often rephrased answers, so exact matches were low.
* The setup worked but can be improved for better results.

## **Recommendations**
* Try using more chunks (`k=2` or `k=4`) for better context.
* Test different methods like `"refine"` or `"map_reduce"` to combine context better.
* Adjust prompts to guide the model more clearly.
